reference.

https://python.langchain.com/docs/expression_language/get_started

In [1]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"



Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

raw_documents = TextLoader('./private_text.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))

retriever = vectorstore.as_retriever()

make a template with context

In [4]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
output_parser = StrOutputParser()


context include custom test resutls fetched by the retriever. 
question will contain user's question.

In [5]:

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | output_parser
)

chain.invoke("tell me about the funnyIO")

'FunnyIO is a high-performance and scalable host controller interface that is designed for accessing solid-state drives (SSDs) over a PCI Express (PCIe) bus. The development of FunnyIO drivers is crucial for enabling efficient communication between the operating system and FunnyIO storage devices. The FunnyIO protocol is specifically designed to leverage the low latency and parallelism offered by modern SSDs, providing efficient command submission and completion mechanisms to enable high-speed data transfers between the host and the storage device.\n\nKey aspects of FunnyIO driver development include a deep understanding of the FunnyIO protocol, PCIe interface, memory management, command queues, error handling, and performance optimization techniques. FunnyIO drivers interact with storage devices over the PCIe interface, requiring knowledge of PCIe bus architecture and communication protocols for efficient driver development.\n\nMemory management is a critical component, with a focus o